## 1. Установка необходимых библиотек, включая MediaPipe и OpenCV:

In [ ]:
!pip install mediapipe opencv-python-headless

In [ ]:
import gdown, os, shutil                                                        # Системные инструменты
from tqdm.auto import tqdm                                                      # Модуль для создания прогресс-бара при выполнении циклов
import numpy as np
import csv
import mediapipe as mp                                                          # Библиотека MediaPipe
import cv2                                                                      # Библиотека обработки изображений

Из-за проблем с распаковкой архивов с файлами в кирилице качаем не архив, а папку с файлами

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#shutil.copytree('/content/drive/MyDrive/ИИ', '/content/ИИ')
shutil.copytree('/content/drive/MyDrive/ИИ/ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)', '/content/ИИ/ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)')

## 2. Создание пустого файл CSV с заголовками содержащими: номера и типы точек x, y, z и visibility.

In [ ]:
with open('dataset.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([ 'correct', 'main_folder', 'folder', 'filename', 'frame#', 'x_0', 'y_0', 'z_0', 'v_0',
                     'x_1', 'y_1', 'z_1', 'v_1', 'x_2', 'y_2', 'z_2', 'v_2',
                     'x_3', 'y_3', 'z_3', 'v_3', 'x_4', 'y_4', 'z_4', 'v_4',
                     'x_5', 'y_5', 'z_5', 'v_5', 'x_6', 'y_6', 'z_6', 'v_6',
                     'x_7', 'y_7', 'z_7', 'v_7', 'x_8', 'y_8', 'z_8', 'v_8',
                     'x_9', 'y_9', 'z_9', 'v_9', 'x_10', 'y_10', 'z_10', 'v_10',
                     'x_11', 'y_11', 'z_11', 'v_11', 'x_12', 'y_12', 'z_12', 'v_12',
                     'x_13', 'y_13', 'z_13', 'v_13', 'x_14', 'y_14', 'z_14', 'v_14',
                     'x_15', 'y_15', 'z_15', 'v_15', 'x_16', 'y_16', 'z_16', 'v_16',
                     'x_17', 'y_17', 'z_17', 'v_17', 'x_18', 'y_18', 'z_18', 'v_18',
                     'x_19', 'y_19', 'z_19', 'v_19', 'x_20', 'y_20', 'z_20', 'v_20',
                     'x_21', 'y_21', 'z_21', 'v_21', 'x_22', 'y_22', 'z_22', 'v_22',
                     'x_23', 'y_23', 'z_23', 'v_23', 'x_24', 'y_24', 'z_24', 'v_24',
                     'x_25', 'y_25', 'z_25', 'v_25', 'x_26', 'y_26', 'z_26', 'v_26',
                     'x_27', 'y_27', 'z_27', 'v_27', 'x_28', 'y_28', 'z_28', 'v_28',
                     'x_29', 'y_29', 'z_29', 'v_29', 'x_30', 'y_30', 'z_30', 'v_30',
                     'x_31', 'y_31', 'z_31', 'v_31', 'x_32', 'y_32', 'z_32', 'v_32'])

## 3. Получаем список всех видеофайлов в папке "ИИ" и ее подпапках

In [ ]:
video_files = []
for root, dirs, files in sorted(os.walk('/content/ИИ')):
    for file in files:
        video_files.append(os.path.join(root, file))
video_files

## 4. Функция обрабатки видеофайла -> записывает скелетные точки в файл CSV.

In [ ]:
mp_pose = mp.solutions.pose                                                     # Создаем объекты для обнаружения скелетных точек на изображении
mp_mask = mp.solutions.holistic                                                 # Создаем объекты для наложение маски на изображении

def process_video(video_file, size=(854, 480)):
    folder = video_file.split(sep='/')[-4:]                                     # Разделяем путь к файлу на части: имя файла и имена папок
    filename_no_ext = os.path.splitext(folder[3])[0]                            # Получаем имя файла без расширения, чтобы использовать его в качестве идентификатора

    pose = mp_pose.Pose(model_complexity=2,                                     # задаем параметры для 'pose'
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)
    mask = mp_mask.Holistic(  model_complexity=2,                               # задаем параметры для 'mask'
                        enable_segmentation=True)

    cap = cv2.VideoCapture(video_file)                                      # Создаем объект cv2.VideoCapture для чтения видео файла
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))                    # Получаем количество кадров в видео файле

    with open('dataset.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        n = 0                                                               # Перменная для нумерации кадров
        for i in tqdm(range(frame_count)):                                  # Пробегаем по каждому кадру в видео файле
            ret, image = cap.read()
            if not ret:                                                     # Если кадр не считался, то прекращаем цикл
                break
            image = cv2.resize(image, (size))                                       # Приводим размер изображения к заданному размеру
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)                  # Переводим картинку в RGB

            results1 = mask.process(image)                                          # С помощью метода process выполняется сегментация на переднем и заднем планах
            #condition = np.stack((results1.segmentation_mask,) * 3, axis=-1) > 0.5 # Булева маска сегментации на 2 канала с порогом 0,5
            if results1.segmentation_mask is not None:
                condition = np.stack((results1.segmentation_mask,) * 3, axis=-1) > 0.5
            else:
                print('В results1.segmentation_mask ошибка')
                results1.segmentation_mask = np.nan_to_num(results1.segmentation_mask, nan=0.0)
                #results1.segmentation_mask = np.zeros_like(results1.segmentation_mask)
                condition = np.stack((results1.segmentation_mask,) * 3, axis=-1) > 0.5

            bg_image = np.zeros(image.shape, dtype=np.uint8)                        # Изображение заднего плана, заполненное нулями
            image = np.where(condition, image, bg_image)                            # Выходное изображение содержит передний план, если значение в маске сегментации равно 1, и задний план, если значение равно 0

            results2 = pose.process(image)                                   # Обрабатываем изображение для обнаружения скелетных точек

            if results2.pose_landmarks is not None:                          # Если скелетные точки обнаружены
                row = [folder[0], folder[1], folder[2], filename_no_ext, n] # Создаем новую строку "row" в CSV файле, содержащую имена папок, имя файла и координаты всех 33 скелетных точек
                for landmark in results2.pose_landmarks.landmark:
                    row.append(landmark.x)
                    row.append(landmark.y)
                    row.append(landmark.z)
                    row.append(landmark.visibility)
                writer.writerow(row)                                        # Записываем строку "row" в CSV файл
                n += 1                                                      # Увеличиваем номер кадра

    cap.release()                                                           # Освобождаем ресурсы, используемые 'cap'
    pose.close()
    mask.close()

## 5. Обрабатываем видеофайлы из списка и записываем скелетные точки в файл CSV

In [ ]:
size = (1280, 720)

for video_file in tqdm(video_files):
    process_video(video_file, size)

Сохраняем получившийся CSV файл на /content/drive/MyDrive

In [ ]:
shutil.copy('/content/dataset.csv', '/content/drive/MyDrive/dataset.csv')